In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We will compute the optical depth given $x_e(z)$, following the convention that $x_e$ is defined with respect to the Hydrogen.  So this means it exceeds unity when He is reionized.  We will also follow the usual assumption that He becomes singly ionized at the same time as H, but doubly ionized at a much lower redshift (when energetic photons are produced in enough numbers by QSOs).

In [ ]:
sigmaT = 6.6524e-29        # SI units.
mass_p = 1.673e-27         # SI units.
Yp     = 0.25              # He mass fraction -- beware Planck redefines as (1/4)number.
ntot4  = 3.9751            # Mass ratio of He/H atoms.
ynum   = Yp/ntot4/(1-Yp)   # Extra number of e per He ionization.

In [ ]:
rhoc   = 1.8784e-26        # SI units, for h=1
Lhub   = 9.252e25          # c/H0 in SI units for h=1
#
OmM = 0.31
hub = 0.6766
wb  = 0.02242
OmB = wb/hub**2
OmL = 1.0-OmM
rhoB= wb * (1-Yp) * rhoc
ne0 = rhoB/mass_p

In [ ]:
zz  = np.linspace(0.0,10.0,1000)
# Just pick some quasi-random forms for now.
xe_H   = 0.5*( 1+np.tanh((8.0-zz)/0.5) )
xe_HeI = 0.5*( 1+np.tanh((8.0-zz)/0.5) )
xe_HeII= 0.5*( 1+np.tanh((3.5-zz)/0.5) )
# Make the full ionization history.
xe = xe_H + ynum*xe_HeI + ynum*xe_HeII
# Plot the contributions for sanity.
fig,ax = plt.subplots(1,1)
ax.plot(zz,xe_H)
ax.plot(zz,xe_HeI)
ax.plot(zz,xe_HeII)
ax.plot(zz,xe)
ax.set_xlabel('Redshift')

In [ ]:
# and compute tau.
zp1 = 1+zz
Ez  = np.sqrt( OmM*zp1**3 + OmL )
dtau= sigmaT*ne0*Lhub*xe*zp1**2/Ez
tau = np.trapz(dtau,x=zz)/hub
print(tau)

Now let us consider the ionization history [proposed by Faucher-Giguere](https://arxiv.org/abs/1903.08657):
$$
  x_e(z) = \left(x_H+x_{HeI}\right)\Theta + x_{HeII}\Theta 
$$
with a 'modified step function', $\Theta$, defined as
$$
  \Theta = \frac{1}{2}\left[1+\sin\left(\frac{\pi[z_0-z]}{2\Delta z}\right) \right]
$$
for $z_0-\Delta z<z<z_0+\Delta z$ and then being $0$ or $1$ outside those limits.  The exact $n_e(z)$ is slightly more complicated than given above leading to $\mathcal{O}(0.01)$ different results for $\tau$, but this will be good enough for our purposes:

In [ ]:
zrei_H ,dz_H  = 7.7,0.25
zrei_He,dz_He = 3.5,0.25
# Set up the contributions, not worrying about clipping \Theta yet.
xe_H   = 0.5*( 1+np.sin(np.pi*(zrei_H -zz)/2/dz_H ) )
xe_HeI = 0.5*( 1+np.sin(np.pi*(zrei_H -zz)/2/dz_H ) )
xe_HeII= 0.5*( 1+np.sin(np.pi*(zrei_He-zz)/2/dz_He) )
# Now clip off the wings of these functions
xe_H[    zz<zrei_H - dz_H ] = 1
xe_H[    zz>zrei_H + dz_H ] = 0
xe_HeI[  zz<zrei_H - dz_H ] = 1
xe_HeI[  zz>zrei_H + dz_H ] = 0
xe_HeII[ zz<zrei_He- dz_He] = 1
xe_HeII[ zz>zrei_He+ dz_He] = 0
# Make the full ionization history.
xe = xe_H + ynum*xe_HeI + ynum*xe_HeII
# Plot it!
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(zz,xe)
ax.set_xlabel('Redshift')

In [ ]:
# and compute tau.
zp1 = 1+zz
Ez  = np.sqrt( OmM*zp1**3 + OmL )
dtau= sigmaT*ne0*Lhub*xe*zp1**2/Ez
tau = np.trapz(dtau,x=zz)/hub
print(tau)